In [2]:
import os
import numpy as np
import pandas as pd
import IPython.display as ipd
import tgt
import math
import sklearn
import librosa
import re
import scipy.io.wavfile
from numpy import unique
from numpy import where
from matplotlib import pyplot as plt
import librosa.display
from sklearn import metrics
%matplotlib inline

In [9]:
def readTextGridUpdate(filename,word,pronunciation_vowel):
    the_list = re.compile(word, re.I)
    print(the_list)
    
    try:
        text_grid= tgt.read_textgrid(filename)
    
    except Exception as FileNotFoundError:
        print("file does not exist")
   
    for tier in text_grid.tiers:
        
        try:
            if(tier.get_annotations_with_text(pattern=the_list, n=0, regex=True)):
                interval1=tier.get_annotations_with_text(pattern=the_list, n=0, regex=True)
                start_time_word=interval1[0].start_time
                end_time_word=interval1[0].end_time
                
            

            elif(tier.get_annotations_with_text(pattern=pronunciation_vowel, n=0, regex=False)):
                interval2=tier.get_annotations_with_text(pattern=pronunciation_vowel, n=0, regex=False)
                if(len(interval2)==1):
                    start_time_pronunciation_vowel=interval2[0].start_time
                    end_time_pronunciation_vowel=interval2[0].end_time
                
                elif(len(interval2)>1):
                    for i in interval2:
                        start_time=i.start_time
                        end_time=i.end_time
                        if(i.start_time==start_time_word or abs(i.start_time-start_time_word)<=2):
                            start_time_pronunciation_vowel=start_time
                            end_time_pronunciation_vowel=end_time

        

        except Exception as e:
            print(e)
    return start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel


In [10]:
def audio_trimming(audio_file,start_time,end_time):
    y1,fs1=librosa.load(audio_file,sr=16000)
   # fs1, y1 = scipy.io.wavfile.read(audio_file)
    #fs1,y1
    print("start and end time is ",start_time,end_time)
    l1=np.array([[start_time,end_time]])
    l1=np.ceil(l1*fs1)
    newWavFileAsList = []
    for elem in l1:
        startRead = elem[0]
        endRead = elem[1]
       
        if startRead >= y1.shape[0]:
            startRead = y1.shape[0]-1
        if endRead >= y1.shape[0]:
            endRead = y1.shape[0]-1
        newWavFileAsList.extend(y1[int(startRead):int(endRead)])

    newWavFile = np.array(newWavFileAsList)
    print(newWavFile.shape)
   # scipy.io.wavfile.write('test5.wav', fs1, newWavFile)
    #print("file saved ")
    return newWavFile
    

In [11]:
def load_displayAudio(filename):
    y,sr=librosa.load(filename,sr=16000)
    print(y.shape)
    plt.figure(figsize=(14, 5))
    librosa.display.waveplot(y, sr=sr)

In [12]:
def toFeature(file_paths,base_path_textGrid,base_path_audio,word,pronunciation_vowel):
    features_word=[]
    features_vowel=[]
    filenames=[]
    SAMPLE_RATE=16000
    i=0

    for path in file_paths:
        filenames.append(path)
        
        try:
            print(i)
            i=i+1
        
            #X, sample_rate = librosa.load(os.path.join(base_path,path),sr=SAMPLE_RATE)
            
            textGrid_file_path=os.path.join(base_path_textGrid,path)+'.TextGrid'
            start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel=readTextGridUpdate(textGrid_file_path,word,pronunciation_vowel)
            
            print(start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
            audio_file_path=os.path.join(base_path_audio,path)+'.wav'
            
            newWavFile_word=audio_trimming(audio_file_path,start_time_word,end_time_word)
            newWavFile_vowel=audio_trimming(audio_file_path,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
            
            sample_rate = np.array(SAMPLE_RATE)
            mfccs_word = np.mean(librosa.feature.mfcc(y=newWavFile_word,sr=sample_rate,n_mfcc=13),axis=1)
            mfccs_vowel = np.mean(librosa.feature.mfcc(y=newWavFile_vowel,sr=sample_rate,n_mfcc=13),axis=1)
            
            features_word.append(mfccs_word)
            features_vowel.append(mfccs_vowel)
            
        except:
            print("Exception is caused because of the file ",path)
            features_word.append()
            features_vowel.append()
            
    df_features_word,df_features_vowel=make_dataframe(filenames,features_word,features_vowel)
    return df_features_word,df_features_vowel

In [13]:
def make_dataframe(filenames,features_word,features_vowel):
    
    df_word=pd.DataFrame({'feature':features_word})
    df_vowel=pd.DataFrame({'feature':features_vowel})
    
    
    
    df_word=df_word.feature.apply(pd.Series)
    df_vowel=df_vowel.feature.apply(pd.Series)
    
    
    df_word['file_name']=filenames
    df_vowel['file_name']=filenames
    
    return df_word,df_vowel
    

In [14]:
if __name__=="__main__":
    # three variables are needed here the two  basepaths and the dataframe path
    # as an additional feature we can definitely ask for the feature to be extracted
    base_path_textGrid="D://Himani-work/gsoc2020/dataset/ideology_five_words_version2/ideology_textGrid_five_dataset_version2/"
    base_path_audio="D://Himani-work/gsoc2020/dataset/ideology_five_words_version2/ideology_wav_five_dataset_version2/"
    
    textGridFiles=list(os.listdir(base_path_textGrid))
    audioFiles=list(os.listdir(base_path_audio))
    all_audio_files=list(pd.DataFrame({'audioFile':audioFiles})['audioFile'].str.split(".wav",expand=True).iloc[:,0])
    all_textGrid_files=list(pd.DataFrame({'textGridFile':textGridFiles})['textGridFile'].str.split(".TextGrid",expand=True).iloc[:,0])
    print("total texGrid and audio files",len(all_audio_files),len(all_textGrid_files))
    
    
    
    final_files=list(set(all_audio_files)&set(all_textGrid_files))
    print("final files",len(final_files))
    
    # Flow is like call the 
    word=r"ideology"
    pronunciation_vowel="aI"


    df_features_word,df_features_vowel=toFeature(final_files,base_path_textGrid,base_path_audio,word,pronunciation_vowel)

    df_features_word.to_csv('mfcc0_ideology_five_features_word.csv',index=False)
    df_features_vowel.to_csv('mfcc0_ideology_five_features_vowel.csv',index=False)

total texGrid and audio files 2921 2887
final files 2887
0
re.compile('ideology', re.IGNORECASE)
1.29 2.06 1.29 1.55
start and end time is  1.29 2.06
(12320,)
start and end time is  1.29 1.55
(4160,)
1
re.compile('ideology', re.IGNORECASE)
1.35 2.12 1.35 1.51
start and end time is  1.35 2.12
(12320,)
start and end time is  1.35 1.51
(2560,)
2
re.compile('ideology', re.IGNORECASE)
1.83 2.51 1.83 1.95
start and end time is  1.83 2.51
(10880,)
start and end time is  1.83 1.95
(1920,)
3
re.compile('ideology', re.IGNORECASE)
1.06 1.56 1.97 2.03
start and end time is  1.06 1.56
(8000,)
start and end time is  1.97 2.03
(960,)
4
re.compile('ideology', re.IGNORECASE)
0.62 1.2 1.84 1.9
start and end time is  0.62 1.2
(9280,)
start and end time is  1.84 1.9
(960,)
5
re.compile('ideology', re.IGNORECASE)
3.14 4.18 3.59 3.69
start and end time is  3.14 4.18
(16640,)
start and end time is  3.59 3.69
(1600,)
6
re.compile('ideology', re.IGNORECASE)
0.99 1.8 1.08 1.21
start and end time is  0.99 1.8
(1

2.79 3.7 2.79 3.0
start and end time is  2.79 3.7
(14560,)
start and end time is  2.79 3.0
(3360,)
60
re.compile('ideology', re.IGNORECASE)
1.51 2.48 1.69 1.8
start and end time is  1.51 2.48
(15520,)
start and end time is  1.69 1.8
(1760,)
61
re.compile('ideology', re.IGNORECASE)
0.7 1.42 0.7 0.82
start and end time is  0.7 1.42
(11520,)
start and end time is  0.7 0.82
(1920,)
62
re.compile('ideology', re.IGNORECASE)
0.54 1.35 2.51 2.7
start and end time is  0.54 1.35
(12960,)
start and end time is  2.51 2.7
(3040,)
63
re.compile('ideology', re.IGNORECASE)
3.31 3.99 4.49 4.72
start and end time is  3.31 3.99
(10880,)
start and end time is  4.49 4.72
(3680,)
64
re.compile('ideology', re.IGNORECASE)
1.4 2.1 1.4 1.63
start and end time is  1.4 2.1
(11200,)
start and end time is  1.4 1.63
(3680,)
65
re.compile('ideology', re.IGNORECASE)
2.4 3.11 2.4 2.45
start and end time is  2.4 3.11
(11360,)
start and end time is  2.4 2.45
(800,)
66
re.compile('ideology', re.IGNORECASE)
1.01 1.9 1.07 1

1.32 2.15 1.32 1.48
start and end time is  1.32 2.15
(13280,)
start and end time is  1.32 1.48
(2560,)
118
re.compile('ideology', re.IGNORECASE)
2.04 2.81 3.24 3.65
start and end time is  2.04 2.81
(12320,)
start and end time is  3.24 3.65
(6560,)
119
re.compile('ideology', re.IGNORECASE)
0.78 1.46 0.78 0.92
start and end time is  0.78 1.46
(10880,)
start and end time is  0.78 0.92
(2240,)
120
re.compile('ideology', re.IGNORECASE)
3.19 3.93 3.19 3.33
start and end time is  3.19 3.93
(11840,)
start and end time is  3.19 3.33
(2240,)
121
re.compile('ideology', re.IGNORECASE)
1.74 2.72 1.89 2.08
start and end time is  1.74 2.72
(15680,)
start and end time is  1.89 2.08
(3040,)
122
re.compile('ideology', re.IGNORECASE)
2.08 3.51 2.2 2.41
start and end time is  2.08 3.51
(22880,)
start and end time is  2.2 2.41
(3360,)
123
re.compile('ideology', re.IGNORECASE)
2.37 3.39 2.54 2.68
start and end time is  2.37 3.39
(16320,)
start and end time is  2.54 2.68
(2240,)
124
re.compile('ideology', re

3.19 4.2 3.3 3.51
start and end time is  3.19 4.2
(16160,)
start and end time is  3.3 3.51
(3360,)
178
re.compile('ideology', re.IGNORECASE)
2.67 3.42 2.67 2.76
start and end time is  2.67 3.42
(12000,)
start and end time is  2.67 2.76
(1440,)
179
re.compile('ideology', re.IGNORECASE)
1.01 2.01 1.01 1.15
start and end time is  1.01 2.01
(16000,)
start and end time is  1.01 1.15
(2240,)
180
re.compile('ideology', re.IGNORECASE)
3.02 4.7 3.17 3.35
start and end time is  3.02 4.7
(26880,)
start and end time is  3.17 3.35
(2880,)
181
re.compile('ideology', re.IGNORECASE)
3.88 4.5 3.88 3.98
start and end time is  3.88 4.5
(9920,)
start and end time is  3.88 3.98
(1600,)
182
re.compile('ideology', re.IGNORECASE)
3.49 4.67 3.49 3.72
start and end time is  3.49 4.67
(18880,)
start and end time is  3.49 3.72
(3680,)
183
re.compile('ideology', re.IGNORECASE)
3.09 3.69 3.09 3.18
start and end time is  3.09 3.69
(9600,)
start and end time is  3.09 3.18
(1440,)
184
re.compile('ideology', re.IGNOREC

start and end time is  0.66 1.22
(8960,)
start and end time is  0.66 0.76
(1600,)
235
re.compile('ideology', re.IGNORECASE)
2.96 3.73 2.96 3.05
start and end time is  2.96 3.73
(12320,)
start and end time is  2.96 3.05
(1440,)
236
re.compile('ideology', re.IGNORECASE)
2.63 3.51 2.71 2.85
start and end time is  2.63 3.51
(14080,)
start and end time is  2.71 2.85
(2240,)
237
re.compile('ideology', re.IGNORECASE)
1.07 1.98 1.14 1.29
start and end time is  1.07 1.98
(14560,)
start and end time is  1.14 1.29
(2400,)
238
re.compile('ideology', re.IGNORECASE)
1.16 2.1 1.16 1.3
start and end time is  1.16 2.1
(15040,)
start and end time is  1.16 1.3
(2240,)
239
re.compile('ideology', re.IGNORECASE)
0.67 1.41 0.67 0.87
start and end time is  0.67 1.41
(11840,)
start and end time is  0.67 0.87
(3200,)
240
re.compile('ideology', re.IGNORECASE)
2.01 2.95 2.11 2.25
start and end time is  2.01 2.95
(15040,)
start and end time is  2.11 2.25
(2240,)
241
re.compile('ideology', re.IGNORECASE)
2.03 2.71 

...

2624
re.compile('ideology', re.IGNORECASE)
0.34 1.2 0.34 0.48
start and end time is  0.34 1.2
(13760,)
start and end time is  0.34 0.48
(2240,)
2625
re.compile('ideology', re.IGNORECASE)
1.73 2.32 1.73 1.81
start and end time is  1.73 2.32
(9440,)
start and end time is  1.73 1.81
(1280,)
2626
re.compile('ideology', re.IGNORECASE)
1.83 2.68 1.83 2.0
start and end time is  1.83 2.68
(13600,)
start and end time is  1.83 2.0
(2720,)
2627
re.compile('ideology', re.IGNORECASE)
1.33 2.03 1.33 1.47
start and end time is  1.33 2.03
(11200,)
start and end time is  1.33 1.47
(2240,)
2628
re.compile('ideology', re.IGNORECASE)
1.95 2.65 1.95 2.09
start and end time is  1.95 2.65
(11200,)
start and end time is  1.95 2.09
(2240,)
2629
re.compile('ideology', re.IGNORECASE)
6.09 6.67 6.09 6.22
start and end time is  6.09 6.67
(9280,)
start and end time is  6.09 6.22
(2080,)
2630
re.compile('ideology', re.IGNORECASE)
1.3 2.03 1.3 1.41
start and end time is  1.3 2.03
(11680,)
start and end time is  1.3 1

1.92 2.64 1.92 2.02
start and end time is  1.92 2.64
(11520,)
start and end time is  1.92 2.02
(1600,)
2684
re.compile('ideology', re.IGNORECASE)
1.96 2.55 1.96 2.06
start and end time is  1.96 2.55
(9440,)
start and end time is  1.96 2.06
(1600,)
2685
re.compile('ideology', re.IGNORECASE)
2.06 3.05 2.19 2.31
start and end time is  2.06 3.05
(15840,)
start and end time is  2.19 2.31
(1920,)
2686
re.compile('ideology', re.IGNORECASE)
2.98 3.6 4.17 4.25
start and end time is  2.98 3.6
(9920,)
start and end time is  4.17 4.25
(1280,)
2687
re.compile('ideology', re.IGNORECASE)
1.72 2.23 1.72 1.87
start and end time is  1.72 2.23
(8160,)
start and end time is  1.72 1.87
(2400,)
2688
re.compile('ideology', re.IGNORECASE)
2.3 2.98 2.3 2.46
start and end time is  2.3 2.98
(10880,)
start and end time is  2.3 2.46
(2560,)
2689
re.compile('ideology', re.IGNORECASE)
2.84 3.73 2.84 2.97
start and end time is  2.84 3.73
(14240,)
start and end time is  2.84 2.97
(2080,)
2690
re.compile('ideology', re

(14560,)
start and end time is  1.27 1.4
(2080,)
2741
re.compile('ideology', re.IGNORECASE)
3.32 3.92 3.32 3.44
start and end time is  3.32 3.92
(9600,)
start and end time is  3.32 3.44
(1920,)
2742
re.compile('ideology', re.IGNORECASE)
1.845 2.58 1.845 1.98
start and end time is  1.845 2.58
(11760,)
start and end time is  1.845 1.98
(2160,)
2743
re.compile('ideology', re.IGNORECASE)
2.49 3.19 2.49 2.58
start and end time is  2.49 3.19
(11200,)
start and end time is  2.49 2.58
(1440,)
2744
re.compile('ideology', re.IGNORECASE)
2.1 2.9 3.67 3.82
start and end time is  2.1 2.9
(12800,)
start and end time is  3.67 3.82
(2400,)
2745
re.compile('ideology', re.IGNORECASE)
1.8 2.45 1.8 1.92
start and end time is  1.8 2.45
(10400,)
start and end time is  1.8 1.92
(1920,)
2746
re.compile('ideology', re.IGNORECASE)
2.03 5.68 3.86 4.43
start and end time is  2.03 5.68
(58400,)
start and end time is  3.86 4.43
(9120,)
2747
re.compile('ideology', re.IGNORECASE)
0.68 1.4 2.655 2.8
start and end time

start and end time is  2.09 2.87
(12480,)
start and end time is  2.17 2.3
(2080,)
2798
re.compile('ideology', re.IGNORECASE)
1.22 1.895 1.22 1.26
start and end time is  1.22 1.895
(10800,)
start and end time is  1.22 1.26
(640,)
2799
re.compile('ideology', re.IGNORECASE)
1.32 1.93 1.93 1.97
start and end time is  1.32 1.93
(9760,)
start and end time is  1.93 1.97
(640,)
2800
re.compile('ideology', re.IGNORECASE)
0.81 1.43 0.81 0.93
start and end time is  0.81 1.43
(9920,)
start and end time is  0.81 0.93
(1920,)
2801
re.compile('ideology', re.IGNORECASE)
1.98 2.63 3.03 3.18
start and end time is  1.98 2.63
(10400,)
start and end time is  3.03 3.18
(2400,)
2802
re.compile('ideology', re.IGNORECASE)
2.01 2.74 2.01 2.09
start and end time is  2.01 2.74
(11680,)
start and end time is  2.01 2.09
(1280,)
2803
re.compile('ideology', re.IGNORECASE)
1.78 3.62 2.93 3.09
start and end time is  1.78 3.62
(29440,)
start and end time is  2.93 3.09
(2560,)
2804
re.compile('ideology', re.IGNORECASE)
4

2857
re.compile('ideology', re.IGNORECASE)
1.04 2.53 1.4 1.53
start and end time is  1.04 2.53
(23840,)
start and end time is  1.4 1.53
(2080,)
2858
re.compile('ideology', re.IGNORECASE)
1.3 2.03 2.4 2.6
start and end time is  1.3 2.03
(11680,)
start and end time is  2.4 2.6
(3200,)
2859
re.compile('ideology', re.IGNORECASE)
1.28 2.12 1.28 1.4
start and end time is  1.28 2.12
(13440,)
start and end time is  1.28 1.4
(1920,)
2860
re.compile('ideology', re.IGNORECASE)
1.21 2.35 1.65 1.81
start and end time is  1.21 2.35
(18240,)
start and end time is  1.65 1.81
(2560,)
2861
re.compile('ideology', re.IGNORECASE)
1.62 2.24 1.62 1.8
start and end time is  1.62 2.24
(9920,)
start and end time is  1.62 1.8
(2880,)
2862
re.compile('ideology', re.IGNORECASE)
2.335 3.02 2.335 2.4
start and end time is  2.335 3.02
(10960,)
start and end time is  2.335 2.4
(1040,)
2863
re.compile('ideology', re.IGNORECASE)
0.46 0.95 0.46 0.53
start and end time is  0.46 0.95
(7840,)
start and end time is  0.46 0.5